In [1]:
from numba.core.extending import get_cython_function_address
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import datetime
import pytz
import pvlib

main_path = "/Users/nataly/opt/AnacondaProjects/SAPIENS/CLEAN/" 
BSRN_path = "/Users/nataly/opt/AnacondaProjects/SAPIENS/BSRNmin/"

ghi =  ['GHIP_SI02pt100',
        'GHIP_SMP11',
        'GHIP_SMP22',
        'GHIP_SPN1',
        'GHIT_SMP22']

gti = ['TGHIP_SI02pt100', 
       'TGHIP_SI02pt100_dirty',
       'TGIP_SMP11_VENT']
      

months = ['2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06', '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12',
          '2019-01', '2019-02', '2019-03', '2019-04', '2019-05', '2019-06', '2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12',
          '2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06', '2020-07', '2020-08', '2020-09', '2020-10', '2020-11', '2020-12',
          '2021-01']

years = ['2018', '2019', '2020', '2021']

dfkt = pd.DataFrame(columns = ghi)
oie  = pd.DataFrame(columns = ghi)


In [2]:
solar_constant = 1366.1


In [3]:
year = years[0]

In [5]:
       dfghi = pd.read_pickle(BSRN_path + year +'ghiFLAG.pkl')
       dfgti = pd.read_pickle(main_path+year+'GTI.pkl')
       eth   = pd.read_pickle(BSRN_path + year + 'ETH.pkl')
       eth.rename(columns= {0: "eth"}, inplace=True)

In [9]:
       dfgti = dfgti.resample('1Min').mean()  # use in case of 1 min span


In [11]:
dfgti.shape

(525201, 3)

In [12]:
dfghi.shape

(525201, 15)

In [13]:
eth.shape

(525201, 1)

In [19]:
dfghi

,PPmax,PPmin,ERmax,ERmin,F_GHIP_SI02pt100,F_GHIP_SMP11,F_GHIP_SMP22,F_GHIP_SPN1,F_GHIT_SMP22,GHIP_SI02pt100,GHIP_SMP11,GHIP_SMP22,GHIP_SPN1,GHIT_SMP22,sumflags
2018-01-01 00:00:00+00:00,NaN,-4,NaN,-2,0,1,0,0,0,0.006883,-2.200000,0.300000,1.375917,-0.006667,NaN
2018-01-01 00:01:00+00:00,NaN,-4,NaN,-2,0,1,0,0,0,0.006883,-2.183333,0.300000,1.518967,0.000000,NaN
2018-01-01 00:02:00+00:00,NaN,-4,NaN,-2,0,1,0,0,0,-0.010817,-2.185000,0.300000,1.609700,0.000000,NaN
2018-01-01 00:03:00+00:00,NaN,-4,NaN,-2,0,1,0,0,0,-0.001967,-2.246667,0.286667,1.485983,-0.100000,NaN
2018-01-01 00:04:00+00:00,NaN,-4,NaN,-2,0,1,0,0,0,0.000000,-2.650000,0.175000,2.041783,-0.235000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 17:16:00+00:00,150.062456,-4,90.049965,-2,4,4,4,4,4,906.233333,926.716667,938.000000,920.800000,935.850000,NaN
2018-12-31 17:17:00+00:00,146.909733,-4,87.527786,-2,4,4,4,4,4,884.666667,905.066667,918.200000,903.383333,915.016667,NaN
2018-12-31 17:18:00+00:00,143.818534,-4,85.054827,-2,4,4,4,4,4,924.616667,945.633333,955.966667,942.683333,953.983333,NaN
2018-12-31 17:19:00+00:00,140.788898,-4,82.631118,-2,4,4,4,4,4,859.636667,879.733333,892.066667,875.116667,888.483333,NaN


In [20]:
fghi = ['F_GHIP_SI02pt100', 
        'F_GHIP_SMP11', 
        'F_GHIP_SMP22', 
        'F_GHIP_SPN1', 
        'F_GHIT_SMP22']

In [29]:
    dfghi['sumflags']= dfghi.iloc[:,4:8].sum(axis=1)

In [30]:
dfghi

,PPmax,PPmin,ERmax,ERmin,F_GHIP_SI02pt100,F_GHIP_SMP11,F_GHIP_SMP22,F_GHIP_SPN1,F_GHIT_SMP22,GHIP_SI02pt100,GHIP_SMP11,GHIP_SMP22,GHIP_SPN1,GHIT_SMP22,sumflags
2018-01-01 00:00:00+00:00,NaN,-4,NaN,-2,0,1,0,0,0,0.006883,-2.200000,0.300000,1.375917,-0.006667,1
2018-01-01 00:01:00+00:00,NaN,-4,NaN,-2,0,1,0,0,0,0.006883,-2.183333,0.300000,1.518967,0.000000,1
2018-01-01 00:02:00+00:00,NaN,-4,NaN,-2,0,1,0,0,0,-0.010817,-2.185000,0.300000,1.609700,0.000000,1
2018-01-01 00:03:00+00:00,NaN,-4,NaN,-2,0,1,0,0,0,-0.001967,-2.246667,0.286667,1.485983,-0.100000,1
2018-01-01 00:04:00+00:00,NaN,-4,NaN,-2,0,1,0,0,0,0.000000,-2.650000,0.175000,2.041783,-0.235000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 17:16:00+00:00,150.062456,-4,90.049965,-2,4,4,4,4,4,906.233333,926.716667,938.000000,920.800000,935.850000,16
2018-12-31 17:17:00+00:00,146.909733,-4,87.527786,-2,4,4,4,4,4,884.666667,905.066667,918.200000,903.383333,915.016667,16
2018-12-31 17:18:00+00:00,143.818534,-4,85.054827,-2,4,4,4,4,4,924.616667,945.633333,955.966667,942.683333,953.983333,16
2018-12-31 17:19:00+00:00,140.788898,-4,82.631118,-2,4,4,4,4,4,859.636667,879.733333,892.066667,875.116667,888.483333,16


In [31]:
dfghi_ok = dfghi[(dfghi['sumflags'] > 0) & (dfghi['sumflags'] < 12)]


In [32]:
dfghi_ok.shape

(218012, 15)

In [33]:
dfghi_ok

,PPmax,PPmin,ERmax,ERmin,F_GHIP_SI02pt100,F_GHIP_SMP11,F_GHIP_SMP22,F_GHIP_SPN1,F_GHIT_SMP22,GHIP_SI02pt100,GHIP_SMP11,GHIP_SMP22,GHIP_SPN1,GHIT_SMP22,sumflags
2018-01-01 00:00:00+00:00,NaN,-4,NaN,-2,0,1,0,0,0,0.006883,-2.200000,0.300000,1.375917,-0.006667,1
2018-01-01 00:01:00+00:00,NaN,-4,NaN,-2,0,1,0,0,0,0.006883,-2.183333,0.300000,1.518967,0.000000,1
2018-01-01 00:02:00+00:00,NaN,-4,NaN,-2,0,1,0,0,0,-0.010817,-2.185000,0.300000,1.609700,0.000000,1
2018-01-01 00:03:00+00:00,NaN,-4,NaN,-2,0,1,0,0,0,-0.001967,-2.246667,0.286667,1.485983,-0.100000,1
2018-01-01 00:04:00+00:00,NaN,-4,NaN,-2,0,1,0,0,0,0.000000,-2.650000,0.175000,2.041783,-0.235000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 15:41:00+00:00,1260.407475,-4,978.325980,-2,2,2,2,2,2,1072.583333,1101.033333,1108.883333,1085.933333,1108.583333,8
2018-12-31 15:42:00+00:00,1222.505158,-4,948.004126,-2,2,2,2,2,2,1073.050000,1101.416667,1109.133333,1083.216667,1108.800000,8
2018-12-31 15:43:00+00:00,1186.032071,-4,918.825657,-2,2,2,2,2,2,1073.583333,1102.250000,1110.200000,1084.000000,1109.683333,8
2018-12-31 15:44:00+00:00,1150.954948,-4,890.763958,-2,2,2,2,2,2,1072.200000,1100.233333,1108.000000,1082.333333,1107.783333,8


In [34]:
dfghi_okk = pd.merge(dfghi_ok, eth, left_index = True, right_index = True, how = 'inner') 

In [35]:
dfghi_okk.shape

(218012, 16)

In [ ]:
dfkt = dfghi_okk[ghi] / dfghi_okk['eth']
dfkt['flag'] = dfghi_okk['sumflags']

In [1]:
dfghi_okk

NameError: name 'dfghi_okk' is not defined

In [4]:
       dfghi = pd.read_pickle(BSRN_path + year +'ghiFLAG.pkl')
       dfghi['sumflags']= dfghi.iloc[:,4:8].sum(axis=1)
       
       dfgti = pd.read_pickle(main_path+year+'GTI.pkl')
       dfgti = dfgti.resample('1Min').mean()  # use in case of 1 min span

       eth   = pd.read_pickle(BSRN_path + year + 'ETH.pkl')
       eth.rename(columns= {0: "eth"}, inplace=True)
       
       dfghi_ok = dfghi[(dfghi['sumflags'] > 0) & (dfghi['sumflags'] < 12)]

In [5]:
dfghi_ok

,PPmax,PPmin,ERmax,ERmin,F_GHIP_SI02pt100,F_GHIP_SMP11,F_GHIP_SMP22,F_GHIP_SPN1,F_GHIT_SMP22,GHIP_SI02pt100,GHIP_SMP11,GHIP_SMP22,GHIP_SPN1,GHIT_SMP22,sumflags
2018-01-01 00:00:00+00:00,NaN,-4,NaN,-2,0,1,0,0,0,0.006883,-2.200000,0.300000,1.375917,-0.006667,1
2018-01-01 00:01:00+00:00,NaN,-4,NaN,-2,0,1,0,0,0,0.006883,-2.183333,0.300000,1.518967,0.000000,1
2018-01-01 00:02:00+00:00,NaN,-4,NaN,-2,0,1,0,0,0,-0.010817,-2.185000,0.300000,1.609700,0.000000,1
2018-01-01 00:03:00+00:00,NaN,-4,NaN,-2,0,1,0,0,0,-0.001967,-2.246667,0.286667,1.485983,-0.100000,1
2018-01-01 00:04:00+00:00,NaN,-4,NaN,-2,0,1,0,0,0,0.000000,-2.650000,0.175000,2.041783,-0.235000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 15:41:00+00:00,1260.407475,-4,978.325980,-2,2,2,2,2,2,1072.583333,1101.033333,1108.883333,1085.933333,1108.583333,8
2018-12-31 15:42:00+00:00,1222.505158,-4,948.004126,-2,2,2,2,2,2,1073.050000,1101.416667,1109.133333,1083.216667,1108.800000,8
2018-12-31 15:43:00+00:00,1186.032071,-4,918.825657,-2,2,2,2,2,2,1073.583333,1102.250000,1110.200000,1084.000000,1109.683333,8
2018-12-31 15:44:00+00:00,1150.954948,-4,890.763958,-2,2,2,2,2,2,1072.200000,1100.233333,1108.000000,1082.333333,1107.783333,8


In [6]:
dfghi_ok = pd.merge(dfghi_ok, eth, left_index = True, right_index = True, how = 'inner')  # check problems with merge - outer = becomes bigger?? is data getting duplicated? inner = is excluding bad?


In [7]:
dfghi_ok[ghi]

,GHIP_SI02pt100,GHIP_SMP11,GHIP_SMP22,GHIP_SPN1,GHIT_SMP22
2018-01-01 00:00:00+00:00,0.006883,-2.200000,0.300000,1.375917,-0.006667
2018-01-01 00:01:00+00:00,0.006883,-2.183333,0.300000,1.518967,0.000000
2018-01-01 00:02:00+00:00,-0.010817,-2.185000,0.300000,1.609700,0.000000
2018-01-01 00:03:00+00:00,-0.001967,-2.246667,0.286667,1.485983,-0.100000
2018-01-01 00:04:00+00:00,0.000000,-2.650000,0.175000,2.041783,-0.235000
...,...,...,...,...,...
2018-12-31 15:41:00+00:00,1072.583333,1101.033333,1108.883333,1085.933333,1108.583333
2018-12-31 15:42:00+00:00,1073.050000,1101.416667,1109.133333,1083.216667,1108.800000
2018-12-31 15:43:00+00:00,1073.583333,1102.250000,1110.200000,1084.000000,1109.683333
2018-12-31 15:44:00+00:00,1072.200000,1100.233333,1108.000000,1082.333333,1107.783333


In [8]:
temp = dfghi_ok[ghi]

In [7]:
dfkt = pd.DataFrame(columns = ghi)


In [8]:
dfkt = dfghi_ok.GHIP_SI02pt100 / dfghi_ok['eth']


In [9]:
dfkt


2018-01-01 00:00:00+00:00    0.000005
2018-01-01 00:01:00+00:00    0.000005
2018-01-01 00:02:00+00:00   -0.000008
2018-01-01 00:03:00+00:00   -0.000001
2018-01-01 00:04:00+00:00    0.000000
                               ...   
2018-12-31 15:41:00+00:00    0.759162
2018-12-31 15:42:00+00:00    0.759492
2018-12-31 15:43:00+00:00    0.759869
2018-12-31 15:44:00+00:00    0.758890
2018-12-31 15:45:00+00:00    0.759716
Length: 218012, dtype: float64

In [10]:
oie_ghi = dfkt[(dfkt>1)] 

In [11]:
oie_ghi

2018-11-15 14:32:00+00:00    1.000851
2018-11-15 14:33:00+00:00    1.014374
2018-11-19 15:31:00+00:00    1.012198
dtype: float64

In [12]:
oie_gti = dfgti[dfgti > solar_constant]


In [17]:
dfgti.shape

(525201, 3)